This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [2]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-15',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [3]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params=params)

Day of Year: 46


c:\ProgramData\Anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [4]:
stream[0].stats.starttime.month

2

In [6]:
def __plot_specgram_stream(stream, params=None, components='Z', stack_type='linear', detrend='mean', dbscale=True, return_fig=True, **kwargs):
    import matplotlib.dates as mdates
    
    if type(components) is str:
        components = [components].split(' ').strip() #Convert to list for compatability
        stream.select(component=components)
    elif type(components) is list:
        traceList = []
        maxStartTime = obspy.UTCDateTime(-1e10) #Go back pretty far (almost 400 years) to start with
        minEndTime = obspy.UTCDateTime(1e10)
        for comp in components:
            tr = stream.select(component=comp).copy()
            traceList.append(tr[0])
            if tr[0].stats.starttime > maxStartTime:
                maxStartTime = tr[0].stats.starttime
            if tr[0].stats.endtime < minEndTime:
                minEndTime = tr[0].stats.endtime

        for tr in traceList:
            tr.trim(starttime=maxStartTime, endtime=minEndTime)
        
        stream = obspy.Stream(traceList)
        stream.stack(group_by='all', npts_tol=200, stack_type=stack_type)  

    fig, ax = plt.subplots()  

    data = stream[0].data
    sample_rate = stream[0].stats.sampling_rate

    if 'title' in kwargs.keys():
        title=kwargs['title']
    else:
        title=stream[0].id

    if 'cmap' in kwargs.keys():
        cmap=kwargs['cmap']
    else:
        cmap='turbo'

    if params is None:
        hvsr_band = [0.4, 40]
    else:
        hvsr_band = params['hvsr_band']

    spec, freqs, times, im = ax.specgram(x=data, Fs=sample_rate, detrend=detrend, scale_by_freq=True, scale='dB')


    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    #ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=5))
    #ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=1))

    print(stream[0].stats)

    sTime = stream[0].stats.starttime

    timeList = []
    for t in stream[0].times():
        timeList.append(sTime + t)

    if '.' in min(timeList).isoformat():
        xmin = mdates.date2num(datetime.datetime.strptime(min(timeList).isoformat(), '%Y-%m-%dT%H:%M:%S.%f'))
        xmax = mdates.date2num(datetime.datetime.strptime(max(timeList).isoformat(), '%Y-%m-%dT%H:%M:%S.%f'))
    else:
        xmin = mdates.date2num(datetime.datetime.strptime(min(timeList).isoformat(), '%Y-%m-%dT%H:%M:%S'))
        xmax = mdates.date2num(datetime.datetime.strptime(max(timeList).isoformat(), '%Y-%m-%dT%H:%M:%S'))
    print(xmin)
    print(xmax)
    return
    #xmin = datetime.datetime.strptime(min(ppsds[anyKey].current_times_used[:-1]).isoformat(), '%Y-%m-%dT%H:%M:%S.%f')
    #xmax = datetime.datetime.strptime(max(ppsds[anyKey].current_times_used[:-1]).isoformat(), '%Y-%m-%dT%H:%M:%S.%f')
    #xmin = mdates.date2num(xmin)
    #xmax = mdates.date2num(xmax)

    tTicks = mdates.MinuteLocator(byminute=range(0,60,5))
    ax.xaxis.set_major_locator(tTicks)
    tTicks_minor = mdates.SecondLocator(bysecond=[0])
    ax.xaxis.set_minor_locator(tTicks_minor)

    tLabels = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(tLabels)
    plt.tick_params(axis='x', labelsize=8)

    day = "{}-{}-{}".format(stream[0].stats.starttime.year, stream[0].stats.starttime.month, stream[0].stats.starttime.day)

    ymin = hvsr_band[0]
    ymax = hvsr_band[1]

    extList = [xmin, xmax, ymin, ymax]
  
    ax = plt.gca()
    fig = plt.gcf()

    freqticks = np.flip(hvsr_dict['x_freqs'][anyKey])
    yminind = np.argmin(np.abs(ymin-freqticks))
    ymaxind = np.argmin(np.abs(ymax-freqticks))
    freqticks = freqticks[yminind:ymaxind]

    #Set up axes, since data is already in semilog
    axy = ax.twinx()
    axy.zorder=0
    ax.zorder=1
    ax.set_facecolor('#ffffff00') #Create transparent background for front axis
    plt.sca(axy)
    im = plt.imshow(psdArr, origin='lower', extent=extList, aspect='auto', interpolation='nearest', **kwargs)
    plt.tick_params(left=False, right=False)
    plt.yticks([], labels='')
    plt.sca(ax)
    if peak_plot:
        ax.hlines(hvsr_dict['Best Peak']['f0'], xmin,xmax, colors='k', linestyles='dashed', alpha=0.5)

    #FreqTicks =np.arange(1,np.round(max(hvsr_dict['x_freqs'][anyKey]),0), 10)
    plt.title(hvsr_dict['input_params']['site']+': Spectrogram')
    plt.xlabel('UTC Time \n'+day)
    
    if colorbar:
        cbar = plt.colorbar(mappable=im)
        cbar.set_label('H/V Ratio')

    plt.ylabel(ylabel)
    #plt.yticks(freqticks)
    plt.semilogy()
    plt.ylim(hvsr_dict['input_params']['hvsr_band'])

    plt.rcParams['figure.dpi'] = 500
    plt.rcParams['figure.figsize'] = (12,4)
    fig.tight_layout()
    plt.show()

    #del im
    #obspy.imaging.spectrogram.spectrogram(data=data, samp_rate=sample_rate, axes=ax, per_lap=0.75, log=True, title=title, dbscale=dbscale, show=False, kwargs=kwargs)
    #ax = stream.spectrogram(samp_rate=sample_rate, axes=ax, per_lap=0.75, log=True, title=title, cmap=cmap, dbscale=dbscale, show=False)
    #plt.gca()
    #ax = ax[0]
    
    plt.ylim(hvsr_band)
    quadmesh = ax.collections[0]
    Z = quadmesh.get_array().data
    ny, nx = quadmesh._coordinates.shape[:2]
    Z = Z.reshape(ny - 1, nx - 1).T

    Y = Z.shape[0]
    y = np.linspace(0, stream[0].stats.sampling_rate / 2, Y)
    YL = np.where(((y > hvsr_band[0]) & (y < hvsr_band[1])))[0]
    Z = Z[YL, :]
    values = np.linspace(Z.min(), Z.max(), 256)

    perClip = 0.2

    minCVal = values.min() + (values.max() - values.min())*perClip
    maxCVal = values.max() - (values.max() - values.min())*perClip 

    quadmesh.set_clim(minCVal, maxCVal)
    cbar = plt.gcf().colorbar(quadmesh)
    cbar.mappable.set_clim(minCVal, maxCVal)

    ax.clear()
    stream.spectrogram(samp_rate=sample_rate, axes=ax, per_lap=0.75, log=True, title=title, cmap=cmap, dbscale=dbscale, show=False, vmin=minCVal, vmax=maxCVal)
    plt.ylim(hvsr_band)
    cbar = plt.gcf().colorbar(ax.collections[0])

    cmap = quadmesh.get_cmap()
    clim = quadmesh.get_clim()
    print(clim)
    print(Z.shape)

    print(stream[0].stats)
    print(stream[0].times())
    displayTimeList = []
    #for t in stream[0].times():
    #    displayTimeList.append(stream[0].stats.starttime + t)


    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Frequency [Hz]') 

    #ax.set_xticks(times)
    plt.show()
    if return_fig:
        return fig, ax
    return

stream = sprit.fetch_data(params=params)
stream = sprit.trim_data(stream, params, export_dir=r"C:\Users\riley\OneDrive\Documents")

fig, ax = __plot_specgram_stream(stream, components=[ "N", 'E'])

ax.set_title("YEY")

Day of Year: 46
         network: AM
         station: RAC84
        location: 00
         channel: 
       starttime: 2023-02-15T17:04:42.080000Z
         endtime: 2023-02-15T17:34:00.000000Z
   sampling_rate: 100.0
           delta: 0.01
            npts: 175793
           calib: 1.0
         _format: MSEED
      processing: ['ObsPy 1.4.0: trim(endtime=None::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2023, 2, 15, 10, 0))', 'ObsPy 1.4.0: trim(endtime=UTCDateTime(2023, 2, 16, 9, 59, 59)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2023, 2, 15, 10, 0))', 'ObsPy 1.4.0: trim(endtime=UTCDateTime(2023, 2, 15, 17, 34)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2023, 2, 15, 17, 4, 42, 80000))']
        response: Channel Response
	From M/S (None) to COUNTS (None)
	Overall Sensitivity: 3.6e+08 defined at 5.000 Hz
	3 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 3.6e+08
		Stage 2: CoefficientsTy

c:\ProgramData\Anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


ValueError: time data '2023-02-15T17:34:00' does not match format '%Y-%m-%dT%H:%M:%S.%f'

In [4]:
params = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01)
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

In [5]:
response = sprit.select_times(input=hvsr_dict)

In [5]:
plt.ion()
sprit.hvplot(hvsr_dict, kind='HVSR', show=True)

In [16]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

#fig, ax = plt.subplots()


fig, ax = sprit.hvplot(hvsr_dict=hvsr_dict, kind='spec', returnfig=True, cmap='turbo')

clickNo = 0    
xWindows = []
pathList = []
windowDrawn = []
winArtist = []
lineArtist = []

def on_click(event, fig=fig, ax=ax):
    if event.button is MouseButton.RIGHT:
        remove_on_right(event)
    if event.button is MouseButton.LEFT:
        draw_boxes(event)

def draw_boxes(event, fig=fig, ax=ax):
    global clickNo
    global x0
    global xWindows
    global pathList
    global windowDrawn
    global winArtist
    global lineArtist

    if event.inaxes!=ax: return
    #y0, y1 = ax.get_ylim()
    y0=0
    y1=50
    
    if clickNo == 0:
        #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
        x0 = event.xdata
        clickNo += 1
        linArt = plt.axvline(x0, y0, y1, linewidth=0.5, color='k', zorder=100)
        lineArtist.append([linArt, linArt])
    else:
        x1 = event.xdata
        clickNo = 0

        path_data = [
            (matplotlib.path.Path.MOVETO, (x0, y0)),
            (matplotlib.path.Path.LINETO, (x1, y0)),
            (matplotlib.path.Path.LINETO, (x1, y1)),
            (matplotlib.path.Path.LINETO, (x0, y1)),
            (matplotlib.path.Path.LINETO, (x0, y0)),
            (matplotlib.path.Path.CLOSEPOLY, (x0, y0)),
        ]

        codes, verts = zip(*path_data)
        path = matplotlib.path.Path(verts, codes)

        x_win = [x0, x1]
        xWindows.append(x_win)
        pathList.append(path)
        windowDrawn.append(False)
        winArtist.append(None)
        [lineArtist[-1].pop()]
        draw_windows(event=event, pathlist=pathList)
        linArt = plt.axvline(x1, y0, y1, color='k',linewidth=0.5, zorder=100)
        lineArtist[-1].append(linArt)
    fig.canvas.draw() 

def draw_windows(event, pathlist, fig=fig, ax=ax):
    for i, p in enumerate(pathList):
        if windowDrawn[i]:
            pass
        else:
            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.75)
            winArt = ax.add_patch(patch)
            windowDrawn[i] = True
            winArtist[i] = winArt
    if event.button is MouseButton.RIGHT:
        fig.canvas.draw() 

def remove_on_right(event, fig=fig, ax=ax, xWindows=xWindows):
    if xWindows is not None:
        for i, xWins in enumerate(xWindows):
            if event.xdata > xWins[0] and event.xdata < xWins[1]:
                xWindows.pop(i)
                pathList.pop(i)
                winArtist[i].remove()
                winArtist.pop(i)
                windowDrawn.pop(i)
                lineArtist[i][0].remove()
                lineArtist[i][1].remove()
                lineArtist.pop(i)
    fig.canvas.draw() 
    #draw_windows(event, pathlist=pathList)

fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()